# First Flask App

In this notebook (and folder in general), I will create my first Flask app. It follows this article:
https://towardsdatascience.com/create-an-api-to-deploy-machine-learning-models-using-flask-and-heroku-67a011800c50

## Create and Pickle the Model

This project isn't about model-building, so the model is not going to be great. In fact, we're just going to do a super simple logistic regression withthe default hyperparameters.

In [3]:
import pandas as pd
import pickle
from sklearn.linear_model import LogisticRegression

In [4]:
url = 'https://raw.githubusercontent.com/elizabethts/deploy-mlm-flask-heroku/master/titanic.csv'
data = pd.read_csv(url)
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
data.dropna(inplace=True)

target = 'Survived'
features = ['Pclass', 'Age', 'SibSp', 'Fare']

X = data[features]
y = data[target]

model = LogisticRegression()
model.fit(X, y)
model.score(X, y)

0.7103825136612022

In [6]:
# pickle the model (saved as a file in the first_flask_app directory)
pickle.dump(model, open('model.pkl', 'wb'))

## Write the Flask App

Flask needs a .py file, named `app.py`, that

1. Loads the model
2. Creates a Flask App
3. Defines a procedure that
    1. Receives Inputs
    2. Uses the model to make a prediction
    3. Returns the model prediction
    
This one will use JSON format for the inputs and outputs. 

The code appears below, but does not need to be run; it is in the app.py file.

I won't lie...I am not sure what some of the code does or why it needs to do it.

In [7]:
import pandas as pd
from flask import Flask, jsonify, request
import pickle

# load model
model = pickle.load(open('model.pkl', 'rb'))

# create app
app = Flask(__name__)

# Routes
@app.route('/', methods=['POST'])

def predict():
	# get the data
	data = request.get_json(force=True)

	# convert data to dataframe
	data.update((x, [y]) for x, y in data.items())
	data_df = pd.DataFrame.from_dict(data)

	# make predictions
	result = model.predict(data_df)

	# send the output
	output = {'results':int(result[0])}

	# return the output
	return jsonify(results=output)

if __name__ == '__main__':
	app.run(port=5000, debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: 1

/Users/johnrizcallah/opt/anaconda3/envs/flaskenv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Test the App

In [1]:
# local url
url = 'http://127.0.0.1:8000'

In [3]:
import requests
import json

# create sample data
sample = {'Pclass':3, 'Age':2, 'SibSp':1, 'Fare':50}
# convert to JSON
sample = json.dumps(sample)

In [5]:
send_request = requests.post(url, sample)
print(send_request)

<Response [200]>


In [6]:
print(send_request.json())

{'results': {'results': 1}}


The model returns are prediction of '1', meaning that the passenger survived.

## Create Procfile

A procfile specifies the commands (procedure) that are executed by a Heroku app. It must be named `Procfile`, with no extension. This one will be very simple:

`web: gunicorn app:app`

## Create requirements.txt

This file contains all the dependencies of the app. The first step is to run this command in the working directory:

`pip freeze > requirements.txt`

